In [2]:
################ change initiate path
%run ../Startup.py

PID number is 23212


The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

# Load chromatin dataframe


Data below can be generated from the notebook as:

[preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict](../../preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict.ipynb)

In [2]:
df_cells = pd.read_csv(r"E:\DNA_analysis\Postanalysis_WT_MoP\selected_all_cells.csv")

In [3]:
df_cells.head()

,rna_experiment,uid,fov_id,cell_id,subclass,chr,hyb,fiberidx,numfiber,z_um,x_um,y_um
0,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,0,1,1,4.147276,3253.588485,-6244.868778
1,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,4,1,1,4.753624,3253.467060,-6243.619909
2,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,5,1,1,5.500038,3254.268482,-6244.335963
3,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,9,1,1,5.381981,3253.031182,-6243.768668
4,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,10,1,1,4.415525,3252.995276,-6243.899623


In [10]:
class_correspondance = {'Gluta':['L2/3 IT', 'L4/5 IT', 'L5 IT', 'L6 IT', 'L5 ET', 'L5/6 NP', 'L6 CT', 'L6b'],
                       'GABA':['Pvalb', 'Sst', 'Sncg', 'Lamp5', 'Vip'], 'Astro':['Astro'], 'Oligo':['Oligo', 'OPC'],
                       'Micro':['Micro'], 'Endo':['Endo', 'Peri', 'SMC', 'VLMC']}
subcls_to_cls = {}
for cls, subcls_list in class_correspondance.items():
    for subcls in subcls_list:
        subcls_to_cls[subcls] = cls

In [7]:
df_cells['majorType'] = df_cells['subclass'].apply(lambda x: subcls_to_cls[x])
df_cells.head()

,rna_experiment,uid,fov_id,cell_id,subclass,chr,hyb,fiberidx,numfiber,z_um,x_um,y_um,majorType
0,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,0,1,1,4.147276,3253.588485,-6244.868778,Oligo
1,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,4,1,1,4.753624,3253.467060,-6243.619909,Oligo
2,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,5,1,1,5.500038,3254.268482,-6244.335963,Oligo
3,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,9,1,1,5.381981,3253.031182,-6243.768668,Oligo
4,20220304,259202492748634617304623818845147108919,0,12,Oligo,chr1,10,1,1,4.415525,3252.995276,-6243.899623,Oligo


# Calculate chromatin size

In [10]:
# prepare arguments
pos_cols = ['z_um', 'x_um', 'y_um']
info_cols = ['uid', 'majorType', 'subclass', 'chr', 'fiberidx']
sel_cols = info_cols + pos_cols

arg_list = []

for (_,_,_), df in tqdm(df_cells.groupby(['uid','chr','fiberidx'])):
    arg_list.append(df[sel_cols].copy())

arg_list[0]

,uid,majorType,subclass,chr,fiberidx,z_um,x_um,y_um
29787278,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,6.923609,399.191452,4067.596867
29787279,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,7.105266,399.373386,4067.844072
29787280,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,7.428478,398.929961,4068.254104
29787281,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,8.313752,399.304615,4068.385576
29787282,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,8.156805,399.674011,4068.130243
...,...,...,...,...,...,...,...,...
29787341,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,6.999900,399.928268,4069.117701
29787342,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,6.384540,399.238562,4069.147023
29787343,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,6.425906,399.182265,4069.186270
29787344,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1,7.666919,398.902880,4068.434342


Load function **chromatin_property_CW** from [chromatin_property_CW](../../functions/chromatin_property_CW.py)

In [12]:
import multiprocessing as mp
from ../../functions import chromatin_property_CW

In [32]:
info_list = [info_cols,]*len(arg_list)

In [39]:
df_list = []
for arg in tqdm(arg_list):
    df_list.append(arg[0])

100%|█████████████████████████████████████████████████████████████████████| 695013/695013 [00:00<00:00, 3252994.06it/s]


In [46]:
with mp.Pool(16) as pool:
    results = pool.starmap(chromatin_property_CW.calculate_rg_multiprocessing, zip(df_list[:],info_list[:]), chunksize=1)

df_rg = pd.concat(results)
df_rg.reset_index(inplace=True, drop=True)
df_rg.head()

,rg,uid,majorType,subclass,chr,fiberidx
0,1.241714,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr1,1
1,1.163226,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr10,1
2,1.888276,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr11,1
3,1.759560,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr12,1
4,1.404082,100002611273212566104762860093574587628,Gluta,L4/5 IT,chr13,1


In [48]:
####### change output path
data_folder = r'chromatin_property'
df_rg_filename = os.path.join(data_folder, 'WT_MOp_Rg.csv')

if not os.path.exists(df_rg_filename):
    df_rg.to_csv(df_rg_filename)